# Tugas 3 Preprocessing 


Pada Tugas 2 diminta untuk mengolah data preprocesing teks atau Pra pemrosesan text Pemrosesan teks melibatkan pembersihan dan penyiapan data teks mentah untuk analisis atau pelatihan model lebih lanjut. Pra-pemrosesan teks yang tepat dapat secara signifikan berdampak pada kinerja dan akurasi model NLP..

Dibuat Oleh:

*   Nama : Willy Chairullah Fauzi Putra
*   NIM : 220411100045
*   Kelas : PPW A

Link Code :

Link Github :

## Persiapan Library

In [28]:
%pip install -q pyspellchecker
%pip install -q Sastrawi

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [29]:
# library awal untuk perhitungan dan pengolahan teks
import numpy as np
import re
import pandas as pd
import pickle

from collections import Counter

# monitoring
from tqdm import tqdm

# library untuk praproses teks
import nltk

# 1. Stop Word
nltk.download('stopwords')
from nltk.corpus import stopwords

# 3. Cek Ejaan Pembakuan kata B.inggris
from spellchecker import SpellChecker
# Inisialisasi SpellChecker untuk bahasa Indonesia
spell = SpellChecker(language='en')

# 4.Stemming
## Indo
nltk.download('wordnet')
# Menggunaakn Sastrawi untuk memproses teks dalam bahasa Indonesia.
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()

## English
from nltk.stem import PorterStemmer, WordNetLemmatizer
stemmer = PorterStemmer()

# 5. Tokenisasi
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\willy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\willy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Praproses Teks

## Membuat Fungsi

### Menghilangkan symbol atau tanda baca

In [30]:
# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

# Menghapus tanda baca, angka, dan simbol
def clean_punct(text):
    clean_spcl = re.compile('[/(){}\[\]\|@,;_]')
    clean_symbol = re.compile('[^0-9a-z]')
    clean_number = re.compile('[0-9]')
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    text = clean_number.sub('', text)
    return text

# Menghaps double atau lebih whitespace
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

# Menghapus stopwords Indonesia
def clean_stopwords_id(text):
    stopword = set(stopwords.words('indonesian'))
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

# Menghapus stopwords Inggris
def clean_stopwords_en(text):
    stopword = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

# Stemming with Sastrawi
def sastrawistemmer(text):
    st = factory.create_stemmer()
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

# Stemming with Porter Stemmer
def porter_stemming(text):
    # Memecah teks menjadi kata-kata
    words = nltk.word_tokenize(text)  # Tokenisasi
    # Melakukan stemming pada setiap kata
    stemmed_words = [stemmer.stem(word) for word in tqdm(words)]
    # Menggabungkan kata-kata yang telah di-stem
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

# Fungsi untuk memperbaiki ejaan dalam bahasa Inggris dengan tqdm
def correct_spelling_english(text):
    if not isinstance(text, str):
        return text  # Mengabaikan nilai non-string
    words = text.split()
    corrected_words = []
    # Menggunakan tqdm untuk menampilkan progres perbaikan ejaan
    for word in tqdm(words, desc="Fix Spell", unit="kata"):
        corrected = spell.correction(word)  # Periksa ejaan kata
        # Jika spell.correction mengembalikan None, gunakan kata aslinya atau string kosong
        if corrected is None:
            corrected_words.append(word)
        else:
            corrected_words.append(corrected)

    corrected_text = ' '.join(corrected_words)
    return corrected_text

# Fungsi untuk melakukan tokenisasi pada teks
def tokenize_text(text):
    if isinstance(text, str):  # Memastikan input adalah string
        return word_tokenize(text)  # Mengembalikan token
    return []  # Kembali ke list kosong jika bukan string


## PTA Trunojoyo Abstrak Indonesia dan English

### Load Data

#### Data PTA

In [31]:
main_df = pd.read_csv('datasets/pta_teknik_informatika.csv', delimiter=',')
main_df.head()

,penulis,judul,abstrak_indo,abstrak_english,pembimbing_pertama,pembimbing_kedua,prodi
0,A.Ubaidillah S.Kom,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,Sistem informasi akademik (SIAKAD) merupaka...,Academic information systems (SIAKAD) is an in...,Budi Setyono M.T,Hermawan S.T,Teknik Informatika
1,"M. Basith Ardianto,",APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,-,"Drs. Budi Soesilo, MT","Koko Joni, ST",Teknik Informatika
2,"Akhmad Suyandi, S.Kom",RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...,Web server adalah sebuah perangkat lunak serve...,Web server is a server software functioning to...,"Drs. Budi Soesilo, M.T","Hermawan, ST, MT",Teknik Informatika
3,Heri Supriyanto,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Scheduling courses in universities is a ...,"Mulaab, S.Si., M.Kom","Firli Irhamni, ST., M.Kom",Teknik Informatika
4,Septian Rahman Hakim,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,As the development of technology existing in t...,"Arik Kurniawati, S.Kom., M.T.","Haryanto, S.T., M.T.",Teknik Informatika


#### Remove data Missing

In [32]:
main_df = main_df[["abstrak_indo", "abstrak_english"]].dropna()
main_df = main_df[main_df["abstrak_indo"].notna() & (main_df["abstrak_indo"] != "-")]
main_df = main_df[main_df["abstrak_english"].notna() & (main_df["abstrak_english"] != "-")]
main_df.isnull().sum()
main_df.head()

,abstrak_indo,abstrak_english
0,Sistem informasi akademik (SIAKAD) merupaka...,Academic information systems (SIAKAD) is an in...
2,Web server adalah sebuah perangkat lunak serve...,Web server is a server software functioning to...
3,Penjadwalan kuliah di Perguruan Tinggi me...,Scheduling courses in universities is a ...
4,Seiring perkembangan teknologi yang ada diduni...,As the development of technology existing in t...
5,Gerak pekerja ada pada game yang memiliki genr...,Workers' movements in the game that has no gen...


#### Lower case

In [ ]:
main_df['lwr_indo'] = main_df['abstrak_indo'].apply(clean_lower)
main_df['lwr_english'] = main_df['abstrak_english'].apply(clean_lower)
casefolding=pd.DataFrame(main_df[["abstrak_indo","lwr_indo","abstrak_english", "lwr_english"]])
casefolding

,abstrak_indo,lwr_indo,abstrak_english,lwr_english
0,Sistem informasi akademik (SIAKAD) merupaka...,sistem informasi akademik (siakad) merupaka...,Academic information systems (SIAKAD) is an in...,academic information systems (siakad) is an in...
2,Web server adalah sebuah perangkat lunak serve...,web server adalah sebuah perangkat lunak serve...,Web server is a server software functioning to...,web server is a server software functioning to...
3,Penjadwalan kuliah di Perguruan Tinggi me...,penjadwalan kuliah di perguruan tinggi me...,Scheduling courses in universities is a ...,scheduling courses in universities is a ...
4,Seiring perkembangan teknologi yang ada diduni...,seiring perkembangan teknologi yang ada diduni...,As the development of technology existing in t...,as the development of technology existing in t...
5,Gerak pekerja ada pada game yang memiliki genr...,gerak pekerja ada pada game yang memiliki genr...,Workers' movements in the game that has no gen...,workers' movements in the game that has no gen...
...,...,...,...,...
853,Investasi saham selama ini memiliki resiko ker...,investasi saham selama ini memiliki resiko ker...,\nInvestment in shares so far has a very large...,\ninvestment in shares so far has a very large...
854,Information Retrieval (IR) merupakan pengambil...,information retrieval (ir) merupakan pengambil...,Information Retrieval (IR) is retrieval of inf...,information retrieval (ir) is retrieval of inf...
855,Klasifikasi citra merupakan proses pengelompok...,klasifikasi citra merupakan proses pengelompok...,Image classification is a process of grouping ...,image classification is a process of grouping ...
856,Identifikasi atribut pejalan kaki merupakan sa...,identifikasi atribut pejalan kaki merupakan sa...,The identification of pedestrian attributes is...,the identification of pedestrian attributes is...


#### Clean Stopwords

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan stopwords
main_df['clean_sw_indo'] = main_df['lwr_indo'].apply(clean_stopwords_id)
main_df['clean_sw_english'] = main_df['lwr_english'].apply(clean_stopwords_en)
stop_words=pd.DataFrame(main_df[['lwr_indo','clean_sw_indo','lwr_english',"clean_sw_english"]])
stop_words

,lwr_indo,clean_sw_indo,lwr_english,clean_sw_english
0,sistem informasi akademik (siakad) merupaka...,sistem informasi akademik (siakad) sistem info...,academic information systems (siakad) is an in...,academic information systems (siakad) informat...
2,web server adalah sebuah perangkat lunak serve...,web server perangkat lunak server berfungsi me...,web server is a server software functioning to...,web server server software functioning receive...
3,penjadwalan kuliah di perguruan tinggi me...,penjadwalan kuliah perguruan kompleks. permasa...,scheduling courses in universities is a ...,scheduling courses universities complex issue....
4,seiring perkembangan teknologi yang ada diduni...,"seiring perkembangan teknologi didunia, muncul...",as the development of technology existing in t...,"development technology existing world, emergin..."
5,gerak pekerja ada pada game yang memiliki genr...,gerak pekerja game memiliki genre rts (real-ti...,workers' movements in the game that has no gen...,workers' movements game genre rts (real-time s...
...,...,...,...,...
853,investasi saham selama ini memiliki resiko ker...,investasi saham memiliki resiko kerugian perge...,\ninvestment in shares so far has a very large...,investment shares far large risk loss due stoc...
854,information retrieval (ir) merupakan pengambil...,information retrieval (ir) pengambilan informa...,information retrieval (ir) is retrieval of inf...,information retrieval (ir) retrieval informati...
855,klasifikasi citra merupakan proses pengelompok...,klasifikasi citra proses pengelompokan piksel ...,image classification is a process of grouping ...,image classification process grouping image pi...
856,identifikasi atribut pejalan kaki merupakan sa...,identifikasi atribut pejalan kaki salah peneli...,the identification of pedestrian attributes is...,identification pedestrian attributes one many ...


#### Menghilangkan symbol atau tanda baca dan whitespace

In [ ]:
main_df['clean_stb_indo'] = main_df['clean_sw_indo'].apply(clean_punct)
main_df['clean_stb_english'] = main_df['clean_sw_english'].apply(clean_punct)
main_df['clean_stb_indo'] = main_df['clean_stb_indo'].apply(_normalize_whitespace)
main_df['clean_stb_english'] = main_df['clean_stb_english'].apply(_normalize_whitespace)
clean_punct_df=pd.DataFrame(main_df[['clean_sw_indo','clean_stb_indo', 'clean_sw_english',"clean_stb_english"]])
clean_punct_df

,clean_sw_indo,clean_stb_indo,clean_sw_english,clean_stb_english
0,sistem informasi akademik (siakad) sistem info...,sistem informasi akademik siakad sistem inform...,academic information systems (siakad) informat...,academic information systems siakad informatio...
2,web server perangkat lunak server berfungsi me...,web server perangkat lunak server berfungsi me...,web server server software functioning receive...,web server server software functioning receive...
3,penjadwalan kuliah perguruan kompleks. permasa...,penjadwalan kuliah perguruan kompleks permasal...,scheduling courses universities complex issue....,scheduling courses universities complex issue ...
4,"seiring perkembangan teknologi didunia, muncul...",seiring perkembangan teknologi didunia muncul ...,"development technology existing world, emergin...",development technology existing world emerging...
5,gerak pekerja game memiliki genre rts (real-ti...,gerak pekerja game memiliki genre rts real tim...,workers' movements game genre rts (real-time s...,workers movements game genre rts real time str...
...,...,...,...,...
853,investasi saham memiliki resiko kerugian perge...,investasi saham memiliki resiko kerugian perge...,investment shares far large risk loss due stoc...,investment shares far large risk loss due stoc...
854,information retrieval (ir) pengambilan informa...,information retrieval ir pengambilan informasi...,information retrieval (ir) retrieval informati...,information retrieval ir retrieval information...
855,klasifikasi citra proses pengelompokan piksel ...,klasifikasi citra proses pengelompokan piksel ...,image classification process grouping image pi...,image classification process grouping image pi...
856,identifikasi atribut pejalan kaki salah peneli...,identifikasi atribut pejalan kaki salah peneli...,identification pedestrian attributes one many ...,identification pedestrian attributes one many ...


##### Cek Ejaan Peter Norvig

###### Dapatkan Corpus

In [36]:
# 1. Mengambil teks dari kolom 'test' dan menggabungkannya menjadi korpus
corpus_string = ' '.join(main_df['clean_stb_indo'].dropna().tolist())
# Nama file yang ingin Anda simpan
file_path = 'datasets/korpus_indo.txt'

# Membuka file dalam mode 'write' ('w') dan menyimpannya
with open(file_path, 'w', encoding='utf-8') as f:
    f.write(corpus_string)

###### Fungsi

In [37]:
# Membaca file .txt untuk membentuk korpus
def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        corpus = file.read()  # Membaca seluruh isi file
    return corpus

# Memuat korpus dari file .txt
file_path = 'datasets/korpus_indo.txt'  # Ganti dengan path file .txt Anda
CORPUS = load_corpus(file_path)

# Membuat fungsi untuk memecah kata dan menghitung frekuensi
def get_word_list(text):
    words = re.findall(r'\w+', text.lower())  # Mengambil kata
    return Counter(words)

# Menghitung frekuensi kata dalam korpus
word_counts = get_word_list(CORPUS)
total_words = sum(word_counts.values())

# Fungsi untuk menghitung probabilitas
def P(word):
    return word_counts[word] / total_words

# Fungsi untuk menghasilkan kemungkinan koreksi
def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits    = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes   = [L + R[1:] for L, R in splits if R]  # Menghapus satu huruf
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]  # Tukar huruf
    replaces  = [L + c + R[1:] for L, R in splits for c in letters if R]  # Mengganti huruf
    inserts   = [L + c + R for L, R in splits for c in letters]  # Menyisipkan huruf
    return set(deletes + transposes + replaces + inserts)

# Fungsi utama untuk memperbaiki ejaan
def correct(word):
    candidates = (edits1(word) or {word})  # Menghasilkan kandidat
    return max(candidates, key=P)  # Memilih kandidat dengan probabilitas tertinggi

# Fungsi untuk memperbaiki ejaan
def correct_spelling_indo(text):
    # Memisahkan kalimat menjadi kata
    words = text.split()
    # Menggunakan tqdm untuk melacak progres
    corrected_words = []

    # Menggunakan tqdm untuk menampilkan progres
    for word in tqdm(words, desc="Memperbaiki ejaan", unit="kata"):
        corrected_words.append(correct(word))

    return ' '.join(corrected_words)


#### Cek Ejaan Pembakuan Kata

In [38]:
# main_df = main_df.head(10) 

In [ ]:
main_df['clean_typo_indo'] = main_df['clean_stb_indo'].apply(correct_spelling_indo)
main_df['clean_typo_english'] = main_df['clean_stb_english'].apply(correct_spelling_english)
clean_spell=pd.DataFrame(main_df[['clean_stb_indo','clean_typo_indo', 'clean_stb_english',"clean_typo_english"]])
clean_spell

,clean_stb_indo,clean_typo_indo,clean_stb_english,clean_typo_english
0,sistem informasi akademik siakad sistem inform...,sistem informasi akademik siakad sistem inform...,academic information systems siakad informatio...,academic information systems salad information...
2,web server perangkat lunak server berfungsi me...,web server perangkat lunak server berfungsi pe...,web server server software functioning receive...,web server server software functioning receive...
3,penjadwalan kuliah perguruan kompleks permasal...,penjadwalan kuliah perguruan kompleks permasal...,scheduling courses universities complex issue ...,scheduling courses universities complex issue ...
4,seiring perkembangan teknologi didunia muncul ...,seiring perkembangan teknologi didunia muncul ...,development technology existing world emerging...,development technology existing world emerging...
5,gerak pekerja game memiliki genre rts real tim...,gerak pekerja game memiliki genre rs real time...,workers movements game genre rts real time str...,workers movements game genre its real time str...
...,...,...,...,...
853,investasi saham memiliki resiko kerugian perge...,investasi saham memiliki resiko kerugian perge...,investment shares far large risk loss due stoc...,investment shares far large risk loss due stoc...
854,information retrieval ir pengambilan informasi...,information retrieval air pengambilan informas...,information retrieval ir retrieval information...,information retrieval ir retrieval information...
855,klasifikasi citra proses pengelompokan piksel ...,klasifikasi citra proses pengelompokan piksel ...,image classification process grouping image pi...,image classification process grouping image pi...
856,identifikasi atribut pejalan kaki salah peneli...,identifikasi atribut pejalan kali salah peneli...,identification pedestrian attributes one many ...,identification pedestrian attributes one many ...


In [ ]:
main_df['stemming_indo'] = main_df['clean_typo_indo'].apply(sastrawistemmer)
main_df['stemming_english'] = main_df['clean_typo_english'].apply(porter_stemming)
stemming=pd.DataFrame(main_df[['clean_typo_indo','stemming_indo', 'clean_typo_english',"stemming_english"]])
stemming

,clean_typo_indo,stemming_indo,clean_typo_english,stemming_english
0,sistem informasi akademik siakad sistem inform...,sistem informasi akademik siakad sistem inform...,academic information systems salad information...,academ inform system salad inform system funct...
2,web server perangkat lunak server berfungsi pe...,web server perangkat lunak server fungsi terim...,web server server software functioning receive...,web server server softwar function receiv help...
3,penjadwalan kuliah perguruan kompleks permasal...,jadwal kuliah guru kompleks masalah variabel t...,scheduling courses universities complex issue ...,schedul cours univers complex issu problem sev...
4,seiring perkembangan teknologi didunia muncul ...,iring kembang teknologi dunia muncul teknologi...,development technology existing world emerging...,develop technolog exist world emerg technolog ...
5,gerak pekerja game memiliki genre rs real time...,gerak kerja game milik genre rs real time stra...,workers movements game genre its real time str...,worker movement game genr it real time strateg...
...,...,...,...,...
853,investasi saham memiliki resiko kerugian perge...,investasi saham milik resiko rugi gera harga s...,investment shares far large risk loss due stoc...,invest share far larg risk loss due stock pric...
854,information retrieval air pengambilan informas...,information retrieval air ambil informasi simp...,information retrieval ir retrieval information...,inform retriev ir retriev inform store documen...
855,klasifikasi citra proses pengelompokan piksel ...,klasifikasi citra proses kelompok piksel citra...,image classification process grouping image pi...,imag classif process group imag pixel certain ...
856,identifikasi atribut pejalan kali salah peneli...,identifikasi atribut pejal kali salah teliti k...,identification pedestrian attributes one many ...,identif pedestrian attribut one mani studi dev...


In [ ]:
main_df['tokenize_indo'] = main_df['stemming_indo'].apply(tokenize_text)
main_df['tokenize_english'] = main_df['stemming_english'].apply(tokenize_text)
tokenize=pd.DataFrame(main_df[['stemming_indo','tokenize_indo','stemming_english', "tokenize_english"]])
tokenize

,stemming_indo,tokenize_indo,stemming_english,tokenize_english
0,sistem informasi akademik siakad sistem inform...,"[sistem, informasi, akademik, siakad, sistem, ...",academ inform system salad inform system funct...,"[academ, inform, system, salad, inform, system..."
2,web server perangkat lunak server fungsi terim...,"[web, server, perangkat, lunak, server, fungsi...",web server server softwar function receiv help...,"[web, server, server, softwar, function, recei..."
3,jadwal kuliah guru kompleks masalah variabel t...,"[jadwal, kuliah, guru, kompleks, masalah, vari...",schedul cours univers complex issu problem sev...,"[schedul, cours, univers, complex, issu, probl..."
4,iring kembang teknologi dunia muncul teknologi...,"[iring, kembang, teknologi, dunia, muncul, tek...",develop technolog exist world emerg technolog ...,"[develop, technolog, exist, world, emerg, tech..."
5,gerak kerja game milik genre rs real time stra...,"[gerak, kerja, game, milik, genre, rs, real, t...",worker movement game genr it real time strateg...,"[worker, movement, game, genr, it, real, time,..."
...,...,...,...,...
853,investasi saham milik resiko rugi gera harga s...,"[investasi, saham, milik, resiko, rugi, gera, ...",invest share far larg risk loss due stock pric...,"[invest, share, far, larg, risk, loss, due, st..."
854,information retrieval air ambil informasi simp...,"[information, retrieval, air, ambil, informasi...",inform retriev ir retriev inform store documen...,"[inform, retriev, ir, retriev, inform, store, ..."
855,klasifikasi citra proses kelompok piksel citra...,"[klasifikasi, citra, proses, kelompok, piksel,...",imag classif process group imag pixel certain ...,"[imag, classif, process, group, imag, pixel, c..."
856,identifikasi atribut pejal kali salah teliti k...,"[identifikasi, atribut, pejal, kali, salah, te...",identif pedestrian attribut one mani studi dev...,"[identif, pedestrian, attribut, one, mani, stu..."


In [42]:
# Simpan DataFrame ke dalam file CSV
main_df.to_csv('datasets/df_pta_preprocessing.csv', index=False)

In [70]:
main_df

,abstrak_indo,abstrak_english,lwr_indo,lwr_english,clean_sw_indo,clean_sw_english,clean_stb_indo,clean_stb_english,clean_typo_indo,clean_typo_english,stemming_indo,stemming_english,tokenize_indo,tokenize_english
0,Sistem informasi akademik (SIAKAD) merupaka...,Academic information systems (SIAKAD) is an in...,sistem informasi akademik (siakad) merupaka...,academic information systems (siakad) is an in...,sistem informasi akademik (siakad) sistem info...,academic information systems (siakad) informat...,sistem informasi akademik siakad sistem inform...,academic information systems siakad informatio...,sistem informasi akademik siakad sistem inform...,academic information systems salad information...,sistem informasi akademik siakad sistem inform...,academ inform system salad inform system funct...,"[sistem, informasi, akademik, siakad, sistem, ...","[academ, inform, system, salad, inform, system..."
2,Web server adalah sebuah perangkat lunak serve...,Web server is a server software functioning to...,web server adalah sebuah perangkat lunak serve...,web server is a server software functioning to...,web server perangkat lunak server berfungsi me...,web server server software functioning receive...,web server perangkat lunak server berfungsi me...,web server server software functioning receive...,web server perangkat lunak server berfungsi pe...,web server server software functioning receive...,web server perangkat lunak server fungsi terim...,web server server softwar function receiv help...,"[web, server, perangkat, lunak, server, fungsi...","[web, server, server, softwar, function, recei..."
3,Penjadwalan kuliah di Perguruan Tinggi me...,Scheduling courses in universities is a ...,penjadwalan kuliah di perguruan tinggi me...,scheduling courses in universities is a ...,penjadwalan kuliah perguruan kompleks. permasa...,scheduling courses universities complex issue....,penjadwalan kuliah perguruan kompleks permasal...,scheduling courses universities complex issue ...,penjadwalan kuliah perguruan kompleks permasal...,scheduling courses universities complex issue ...,jadwal kuliah guru kompleks masalah variabel t...,schedul cours univers complex issu problem sev...,"[jadwal, kuliah, guru, kompleks, masalah, vari...","[schedul, cours, univers, complex, issu, probl..."
4,Seiring perkembangan teknologi yang ada diduni...,As the development of technology existing in t...,seiring perkembangan teknologi yang ada diduni...,as the development of technology existing in t...,"seiring perkembangan teknologi didunia, muncul...","development technology existing world, emergin...",seiring perkembangan teknologi didunia muncul ...,development technology existing world emerging...,seiring perkembangan teknologi didunia muncul ...,development technology existing world emerging...,iring kembang teknologi dunia muncul teknologi...,develop technolog exist world emerg technolog ...,"[iring, kembang, teknologi, dunia, muncul, tek...","[develop, technolog, exist, world, emerg, tech..."
5,Gerak pekerja ada pada game yang memiliki genr...,Workers' movements in the game that has no gen...,gerak pekerja ada pada game yang memiliki genr...,workers' movements in the game that has no gen...,gerak pekerja game memiliki genre rts (real-ti...,workers' movements game genre rts (real-time s...,gerak pekerja game memiliki genre rts real tim...,workers movements game genre rts real time str...,gerak pekerja game memiliki genre rs real time...,workers movements game genre its real time str...,gerak kerja game milik genre rs real time stra...,worker movement game genr it real time strateg...,"[gerak, kerja, game, milik, genre, rs, real, t...","[worker, movement, game, genr, it, real, time,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,Investasi saham selama ini memiliki resiko ker...,\nInvestment in shares so far has a very large...,investasi saham selama ini memiliki resiko ker...,\ninvestment in shares so far has a very large...,investasi saham me

## PTA Berita Online Kompas.com

### Load Data

#### Data PTA

In [43]:
main_df_berita = pd.read_csv('datasets/data_berita.csv', delimiter=',')
main_df_berita.head()

,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,16 Aktivis yang Ditahan di Polda Metro Jaya Mo...,"JAKARTA, KOMPAS.com - Sebanyak 16 aktivis yang...",17/09/2025,NEWS
1,Erick Thohir Fokus Bangun Kejayaan Olahraga Us...,KOMPAS.com - Erick Thohir resmi dilantik Presi...,17/09/2025,BOLA
2,BMKG: Waspada Banjir Rob di Pesisir Utara Jate...,"SEMARANG, KOMPAS.com - Stasiun Meteorologi Mar...",17/09/2025,REGIONAL
3,"VW ID. Buzz, Transformasi Kombi ke Era Baru","JAKARTA, KOMPAS.com – Volkswagen ID. Buzz menj...",17/09/2025,OTOMOTIF
4,IHSG Cetak Rekor Tertinggi Sepanjang Sejarah p...,"JAKARTA, KOMPAS.com - Indeks Harga Saham Gabun...",17/09/2025,MONEY


#### Remove data Missing

In [44]:
main_df_berita = main_df_berita[["Isi Berita"]].dropna()
main_df_berita.isnull().sum()
main_df_berita.head()

,Isi Berita
0,"JAKARTA, KOMPAS.com - Sebanyak 16 aktivis yang..."
1,KOMPAS.com - Erick Thohir resmi dilantik Presi...
2,"SEMARANG, KOMPAS.com - Stasiun Meteorologi Mar..."
3,"JAKARTA, KOMPAS.com – Volkswagen ID. Buzz menj..."
4,"JAKARTA, KOMPAS.com - Indeks Harga Saham Gabun..."


#### Lower case

In [ ]:
main_df_berita['lwr_indo'] = main_df_berita['Isi Berita'].apply(clean_lower)
casefolding=pd.DataFrame(main_df_berita[["Isi Berita","lwr_indo"]])
casefolding

,Isi Berita,lwr_indo
0,"JAKARTA, KOMPAS.com - Sebanyak 16 aktivis yang...","jakarta, kompas.com - sebanyak 16 aktivis yang..."
1,KOMPAS.com - Erick Thohir resmi dilantik Presi...,kompas.com - erick thohir resmi dilantik presi...
2,"SEMARANG, KOMPAS.com - Stasiun Meteorologi Mar...","semarang, kompas.com - stasiun meteorologi mar..."
3,"JAKARTA, KOMPAS.com – Volkswagen ID. Buzz menj...","jakarta, kompas.com – volkswagen id. buzz menj..."
4,"JAKARTA, KOMPAS.com - Indeks Harga Saham Gabun...","jakarta, kompas.com - indeks harga saham gabun..."
...,...,...
100,"JAKARTA, KOMPAS.com – Gubernur DKI Jakarta Pra...","jakarta, kompas.com – gubernur dki jakarta pra..."
101,"JAKARTA, KOMPAS.com – Bagi sejumlah pengendara...","jakarta, kompas.com – bagi sejumlah pengendara..."
102,KOMPAS.com - Presiden Prabowo Subianto melanti...,kompas.com - presiden prabowo subianto melanti...
103,"JAKARTA, KOMPAS.com - Presiden Prabowo Subian...","jakarta, kompas.com - presiden prabowo subian..."


#### Clean Stopwords

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan stopwords
main_df_berita['clean_sw_indo'] = main_df_berita['lwr_indo'].apply(clean_stopwords_id)
stop_words = pd.DataFrame(main_df_berita[['lwr_indo','clean_sw_indo']])
stop_words

,lwr_indo,clean_sw_indo
0,"jakarta, kompas.com - sebanyak 16 aktivis yang...","jakarta, kompas.com - 16 aktivis ditahan polda..."
1,kompas.com - erick thohir resmi dilantik presi...,kompas.com - erick thohir resmi dilantik presi...
2,"semarang, kompas.com - stasiun meteorologi mar...","semarang, kompas.com - stasiun meteorologi mar..."
3,"jakarta, kompas.com – volkswagen id. buzz menj...","jakarta, kompas.com – volkswagen id. buzz sala..."
4,"jakarta, kompas.com - indeks harga saham gabun...","jakarta, kompas.com - indeks harga saham gabun..."
...,...,...
100,"jakarta, kompas.com – gubernur dki jakarta pra...","jakarta, kompas.com – gubernur dki jakarta pra..."
101,"jakarta, kompas.com – bagi sejumlah pengendara...","jakarta, kompas.com – pengendara, bensin spbu ..."
102,kompas.com - presiden prabowo subianto melanti...,kompas.com - presiden prabowo subianto melanti...
103,"jakarta, kompas.com - presiden prabowo subian...","jakarta, kompas.com - presiden prabowo subiant..."


#### Menghilangkan symbol atau tanda baca

In [ ]:
main_df_berita['clean_stb_indo'] = main_df_berita['clean_sw_indo'].apply(clean_punct)
clean_punct_df = pd.DataFrame(main_df_berita[['clean_sw_indo','clean_stb_indo']])
clean_punct_df

,clean_sw_indo,clean_stb_indo
0,"jakarta, kompas.com - 16 aktivis ditahan polda...",jakarta kompas com aktivis ditahan polda me...
1,kompas.com - erick thohir resmi dilantik presi...,kompas com erick thohir resmi dilantik presi...
2,"semarang, kompas.com - stasiun meteorologi mar...",semarang kompas com stasiun meteorologi mari...
3,"jakarta, kompas.com – volkswagen id. buzz sala...",jakarta kompas com volkswagen id buzz salah...
4,"jakarta, kompas.com - indeks harga saham gabun...",jakarta kompas com indeks harga saham gabung...
...,...,...
100,"jakarta, kompas.com – gubernur dki jakarta pra...",jakarta kompas com gubernur dki jakarta pram...
101,"jakarta, kompas.com – pengendara, bensin spbu ...",jakarta kompas com pengendara bensin spbu sw...
102,kompas.com - presiden prabowo subianto melanti...,kompas com presiden prabowo subianto melanti...
103,"jakarta, kompas.com - presiden prabowo subiant...",jakarta kompas com presiden prabowo subianto...


##### Cek Ejaan Peter Norvig

###### Fungsi

In [49]:
# Spell checker (default bahasa Inggris, kita isi dengan kosakata dataset CNN)
spell = SpellChecker(language=None)
unique_words = set(" ".join(main_df_berita["clean_stb_indo"].astype(str)).split())
print(unique_words)
spell.word_frequency.load_words(unique_words)

def spellcheck_text(text):
    if pd.isna(text):
        return ""
    tokens = text.split()
    corrected_tokens = []
    for token in tokens:
        if token not in spell:
            correction = spell.correction(token)
            corrected_tokens.append(correction if correction else token)
        else:
            corrected_tokens.append(token)
    return " ".join(corrected_tokens)

{'memproses', 'darurat', 'copper', 'pemanfaatan', 'menempati', 'duduak', 'warnanya', 'paksa', 'amanah', 'kmjam', 'panen', 'menghadiri', 'penutupan', 'sintetis', 'kualitas', 'dukungan', 'direnovasi', 'swt', 'keberadaan', 'memercayakan', 'aspirasi', 'politis', 'januari', 'mengaku', 'artikel', 'reka', 'tenang', 'berubah', 'berlebihan', 'mengenal', 'mengakui', 'dibagi', 'ihsg', 'tampilan', 'basharahu', 'temukan', 'metode', 'bercerai', 'hikam', 'pohon', 'memakan', 'babi', 'kumpulkan', 'jenguk', 'sidang', 'palmerah', 'kesinambungan', 'ikan', 'kapasitas', 'ketidakpuasan', 'dewasa', 'kijang', 'rg', 'asupan', 'soerakoesoemah', 'pengalihan', 'cinta', 'sjafrie', 'manado', 'panas', 'memanggilnya', 'kosmetik', 'irigasi', 'overstay', 'qoumas', 'jaga', 'tekanan', 'melingkar', 'sousa', 'itikad', 'erat', 'pm', 'stoknya', 'nanti', 'data', 'inclusive', 'kisah', 'buffer', 'psikologis', 'tabur', 'lestari', 'terkait', 'menghargai', 'jambon', 'lantik', 'susiati', 'mitra', 'lupa', 'politikus', 'menetapkan', '

#### Cek Ejaan Pembakuan Kata

In [50]:
# main_df_berita = main_df_berita.head(10)

In [ ]:
main_df_berita['clean_typo_indo'] = main_df_berita['clean_stb_indo'].apply(spellcheck_text)
clean_spell=pd.DataFrame(main_df_berita[["clean_stb_indo",'clean_typo_indo']])
clean_spell

,clean_stb_indo,clean_typo_indo
0,jakarta kompas com aktivis ditahan polda me...,jakarta kompas com aktivis ditahan polda metro...
1,kompas com erick thohir resmi dilantik presi...,kompas com erick thohir resmi dilantik preside...
2,semarang kompas com stasiun meteorologi mari...,semarang kompas com stasiun meteorologi mariti...
3,jakarta kompas com volkswagen id buzz salah...,jakarta kompas com volkswagen id buzz salah mo...
4,jakarta kompas com indeks harga saham gabung...,jakarta kompas com indeks harga saham gabungan...
...,...,...
100,jakarta kompas com gubernur dki jakarta pram...,jakarta kompas com gubernur dki jakarta pramon...
101,jakarta kompas com pengendara bensin spbu sw...,jakarta kompas com pengendara bensin spbu swas...
102,kompas com presiden prabowo subianto melanti...,kompas com presiden prabowo subianto melantik ...
103,jakarta kompas com presiden prabowo subianto...,jakarta kompas com presiden prabowo subianto m...


In [ ]:
main_df_berita['stemming_indo'] = main_df_berita['clean_typo_indo'].apply(sastrawistemmer)
stemming=pd.DataFrame(main_df_berita[['clean_typo_indo','stemming_indo']])
stemming

,clean_typo_indo,stemming_indo
0,jakarta kompas com aktivis ditahan polda metro...,jakarta kompas com aktivis tahan polda metro j...
1,kompas com erick thohir resmi dilantik preside...,kompas com erick thohir resmi lantik presiden ...
2,semarang kompas com stasiun meteorologi mariti...,semarang kompas com stasiun meteorologi mariti...
3,jakarta kompas com volkswagen id buzz salah mo...,jakarta kompas com volkswagen id buzz salah mo...
4,jakarta kompas com indeks harga saham gabungan...,jakarta kompas com indeks harga saham gabung i...
...,...,...
100,jakarta kompas com gubernur dki jakarta pramon...,jakarta kompas com gubernur dki jakarta pramon...
101,jakarta kompas com pengendara bensin spbu swas...,jakarta kompas com kendara bensin spbu swasta ...
102,kompas com presiden prabowo subianto melantik ...,kompas com presiden prabowo subianto lantik an...
103,jakarta kompas com presiden prabowo subianto m...,jakarta kompas com presiden prabowo subianto l...


In [ ]:
main_df_berita['tokenize_indo'] = main_df_berita['stemming_indo'].apply(tokenize_text)
tokenize=pd.DataFrame(main_df_berita[['stemming_indo','tokenize_indo']])
tokenize

,stemming_indo,tokenize_indo
0,jakarta kompas com aktivis tahan polda metro j...,"[jakarta, kompas, com, aktivis, tahan, polda, ..."
1,kompas com erick thohir resmi lantik presiden ...,"[kompas, com, erick, thohir, resmi, lantik, pr..."
2,semarang kompas com stasiun meteorologi mariti...,"[semarang, kompas, com, stasiun, meteorologi, ..."
3,jakarta kompas com volkswagen id buzz salah mo...,"[jakarta, kompas, com, volkswagen, id, buzz, s..."
4,jakarta kompas com indeks harga saham gabung i...,"[jakarta, kompas, com, indeks, harga, saham, g..."
...,...,...
100,jakarta kompas com gubernur dki jakarta pramon...,"[jakarta, kompas, com, gubernur, dki, jakarta,..."
101,jakarta kompas com kendara bensin spbu swasta ...,"[jakarta, kompas, com, kendara, bensin, spbu, ..."
102,kompas com presiden prabowo subianto lantik an...,"[kompas, com, presiden, prabowo, subianto, lan..."
103,jakarta kompas com presiden prabowo subianto l...,"[jakarta, kompas, com, presiden, prabowo, subi..."


In [54]:
# Simpan DataFrame ke dalam file CSV
main_df_berita.to_csv('datasets/df_berita_preprocessing.csv', index=False)

In [69]:
main_df_berita

,Isi Berita,lwr_indo,clean_sw_indo,clean_stb_indo,clean_typo_indo,stemming_indo,tokenize_indo
0,"JAKARTA, KOMPAS.com - Sebanyak 16 aktivis yang...","jakarta, kompas.com - sebanyak 16 aktivis yang...","jakarta, kompas.com - 16 aktivis ditahan polda...",jakarta kompas com aktivis ditahan polda me...,jakarta kompas com aktivis ditahan polda metro...,jakarta kompas com aktivis tahan polda metro j...,"[jakarta, kompas, com, aktivis, tahan, polda, ..."
1,KOMPAS.com - Erick Thohir resmi dilantik Presi...,kompas.com - erick thohir resmi dilantik presi...,kompas.com - erick thohir resmi dilantik presi...,kompas com erick thohir resmi dilantik presi...,kompas com erick thohir resmi dilantik preside...,kompas com erick thohir resmi lantik presiden ...,"[kompas, com, erick, thohir, resmi, lantik, pr..."
2,"SEMARANG, KOMPAS.com - Stasiun Meteorologi Mar...","semarang, kompas.com - stasiun meteorologi mar...","semarang, kompas.com - stasiun meteorologi mar...",semarang kompas com stasiun meteorologi mari...,semarang kompas com stasiun meteorologi mariti...,semarang kompas com stasiun meteorologi mariti...,"[semarang, kompas, com, stasiun, meteorologi, ..."
3,"JAKARTA, KOMPAS.com – Volkswagen ID. Buzz menj...","jakarta, kompas.com – volkswagen id. buzz menj...","jakarta, kompas.com – volkswagen id. buzz sala...",jakarta kompas com volkswagen id buzz salah...,jakarta kompas com volkswagen id buzz salah mo...,jakarta kompas com volkswagen id buzz salah mo...,"[jakarta, kompas, com, volkswagen, id, buzz, s..."
4,"JAKARTA, KOMPAS.com - Indeks Harga Saham Gabun...","jakarta, kompas.com - indeks harga saham gabun...","jakarta, kompas.com - indeks harga saham gabun...",jakarta kompas com indeks harga saham gabung...,jakarta kompas com indeks harga saham gabungan...,jakarta kompas com indeks harga saham gabung i...,"[jakarta, kompas, com, indeks, harga, saham, g..."
...,...,...,...,...,...,...,...
100,"JAKARTA, KOMPAS.com – Gubernur DKI Jakarta Pra...","jakarta, kompas.com – gubernur dki jakarta pra...","jakarta, kompas.com – gubernur dki jakarta pra...",jakarta kompas com gubernur dki jakarta pram...,jakarta kompas com gubernur dki jakarta pramon...,jakarta kompas com gubernur dki jakarta pramon...,"[jakarta, kompas, com, gubernur, dki, jakarta,..."
101,"JAKARTA, KOMPAS.com – Bagi sejumlah pengendara...","jakarta, kompas.com – bagi sejumlah pengendara...","jakarta, kompas.com – pengendara, bensin spbu ...",jakarta kompas com pengendara bensin spbu sw...,jakarta kompas com pengendara bensin spbu swas...,jakarta kompas com kendara bensin spbu swasta ...,"[jakarta, kompas, com, kendara, bensin, spbu, ..."
102,KOMPAS.com - Presiden Prabowo Subianto melanti...,kompas.com - presiden prabowo subianto melanti...,kompas.com - presiden prabowo subianto melanti...,kompas com presiden prabowo subianto melanti...,kompas com presiden prabowo subianto melantik ...,kompas com presiden prabowo subianto lantik an...,"[kompas, com, presiden, prabowo, subianto, lan..."
103,"JAKARTA, KOMPAS.com - Presiden Prabowo Subian...","jakarta, kompas.com - presiden prabowo subian...","jakarta, kompas.com - presiden prabowo subiant...",jakarta kompas com presiden prabowo subianto...,jakarta kompas com presiden prabowo subianto m...,jakarta kompas com presiden prabowo subianto l...,"[jakarta, kompas, com, presiden, prabowo, subi..."
